In [139]:
using JuMP
using Gurobi

arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
# paths = []

# numArcs = 10 #length(arcs[:,1])
# numPaths = 20

# #Arc-Path Matrix: 1 if arc is on path and 0 o.w.
# P = Array{Int64}(undef,numArcs,0)
# println(typeof(P))
# println(P)
# println("numPaths = ", numPaths)
# for i = 1:numPaths
#     genPath = rand(0:1, numArcs)
#     P = hcat(P,genPath)
# end
# for i = 1:numArcs
#    println("\t", P[i,:]) 
# end


P = [1 1 0 0; #(1,2)
    0 0 1 0; #(1,3)
    0 0 0 1; #(1,4)
    1 0 0 0; #(2,3)
    0 1 0 0; #(2,5)
    1 0 1 0; #(3,5)
    0 0 0 1 #(4,5)
]

numArcs = 7
numPaths = 4

b_x = 5
d = rand(1:10,numArcs)

println("Interdiction budget = ", b_x)
println("Interdiction cost = ", d)

Interdiction budget = 5
Interdiction cost = [6, 5, 5, 8, 5, 1, 8]


In [141]:
#Formulation 12
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

m = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(m, x[1:numArcs], Bin)
@variable(m, y[1:numArcs]>=0)
@variable(m, u>=0)
for p = 1:numPaths
    @constraint(m, u <= sum((x[a] + y[a])*P[a,p] for a = 1:numArcs))
end
@constraint(m, sum(d[a]*x[a] for a = 1:numArcs) <= b_x)
@constraint(m, sum(y[a] for a = 1:numArcs) == 1)
@objective(m, Max, u)
println(m)
optimize!(m)
println("x = ", findall(JuMP.value.(x).==1))
println("y = ", findall(JuMP.value.(y).>0), " ; ", JuMP.value.(y))
# end

Max u
Subject to
 y[1] + y[2] + y[3] + y[4] + y[5] + y[6] + y[7] == 1.0
 -x[1] - x[4] - x[6] - y[1] - y[4] - y[6] + u <= 0.0
 -x[1] - x[5] - y[1] - y[5] + u <= 0.0
 -x[2] - x[6] - y[2] - y[6] + u <= 0.0
 -x[3] - x[7] - y[3] - y[7] + u <= 0.0
 6 x[1] + 5 x[2] + 5 x[3] + 8 x[4] + 5 x[5] + x[6] + 8 x[7] <= 5.0
 y[1] >= 0.0
 y[2] >= 0.0
 y[3] >= 0.0
 y[4] >= 0.0
 y[5] >= 0.0
 y[6] >= 0.0
 y[7] >= 0.0
 u >= 0.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary

x = [3]
y = [1, 6] ; [0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0]


In [127]:
#Formulation 18
numNodes = maximum(arcs[:,2])
nodes = collect(1:numNodes)
println(nodes)
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

#DUALS SIGN: >= (+) AND <= (-)

# function SP(x)
#     global arcs, numArcs, numPaths, P
#     global nodes, numNodes
#     SP = Model(() -> Gurobi.Optimizer(gurobi_env))
#     π_cons = Array{JuMP.ConstraintRef}(undef, numNodes)
#     β_cons = Array{JuMP.ConstraintRef}(undef, numArcs)
# #     xcap_cons = Array{JuMP.ConstraintRef}(undef, arcNum)
    
#     @variable(SP, u>=0)
#     @variable(SP, y[1:numArcs]>=0)
#     outgoing_s = findall(arcs[:,1].== 1)
#     π_cons[1] = @constraint(SP, sum(y[a] for a in outgoing_s) == 1)
#     for n = 1:numNodes
#         incoming = findall(arcs[:,2].== n)
#         outgoing = findall(arcs[:,1].== n)
#         if n != 1 && n!=5
#             @constraint(SP, sum(y[a] for a in outgoing) - sum(y[a] for a in incoming) == 0)
#         end
#     end
#     for a = 1:numArcs
#         @constraint(SP, u >= y[a])
#         β_cons[a] = @constraint(SP, - y[a] >= x[a] - 1)
#     end
#     @objective(SP, Min, u)
#     optimize!(SP)
# #     println(SP)
# #     println("Solved SP")
# #     println("y = ", JuMP.value.(y))
#     π_s = JuMP.dual(π_cons[1])
#     β = zeros(numArcs)
#     for a = 1:numArcs
#        β[a] = JuMP.dual(β_cons[a])
#     end
#     return JuMP.objective_value.(SP), JuMP.value.(y),π_s, β
# end

function DSP(x)
    global arcs, numArcs, numPaths, P
    global nodes, numNodes
    DSP = Model(() -> Gurobi.Optimizer(gurobi_env))
    @variable(DSP, π[1:numNodes])
    @variable(DSP, α[1:numArcs]>=0)
    @variable(DSP, β[1:numArcs]>=0) 
    for a = 1:numArcs
        i = arcs[a,1]
        j = arcs[a,2]
        if j != 5
            @constraint(DSP, π[i] - π[j] - α[a] - β[a] <= 0)
        else
            @constraint(DSP, π[i] - α[a] - β[a] <= 0)
        end
    end
    @constraint(DSP, sum(α[a] for a = 1:numArcs) <= 1)
    @objective(DSP, Max, π[1] + sum((x[a]-1)*β[a] for a = 1:numArcs))
    println("DSP")
#     println(DSP)
    optimize!(DSP)
    println(JuMP.objective_value.(DSP))
    return JuMP.value.(π[1]), JuMP.value.(β), JuMP.objective_value.(DSP)
end

MP = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(MP, 1e6>=w >=0)
@variable(MP, x[1:numArcs], Bin)
@constraint(MP, sum(d[a]*x[a] for a = 1:numArcs) <= b_x)
@objective(MP, Max, w) # + (b_x - sum(d[a]*x[a] for a = 1:numArcs))/(10^6) )
println(MP)
for iter = 1:3 #Temporary number of iterations
    optimize!(MP)
    x_now = JuMP.value.(x)
    x_now[1] = 1
    println("\nObj func = ",JuMP.objective_value.(MP))
    println("Current x = ", findall(x_now.==1)) 
    π_s, β, DSP_Obj = DSP(x_now)
#     SP_Obj, y, π_s, β = SP(x_now)
    println("π = ", π_s)
    println("β = ", β)
    con = @constraint(MP, w <=  π_s + sum((x[a]-1)*β[a] for a = 1:numArcs)) 
    println(con)
#     println("Current y = ", findall(JuMP.value.(y).==1))
end
println("x = ", JuMP.value.(x))
println("y = ", JuMP.value.(y))

[1, 2, 3, 4, 5]
Max w
Subject to
 2 x[1] + 9 x[2] + 5 x[3] + 3 x[4] + 5 x[5] + 4 x[6] + 2 x[7] <= 10.0
 w >= 0.0
 w <= 1.0e6
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary


Obj func = 1.0e6
Current x = [1]
DSP
0.5
π = 0.5
β = [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
w - 0.5 x[1] <= 0.0

Obj func = 0.5
Current x = [1]
DSP
0.5
π = 0.5
β = [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
w - 0.5 x[1] <= 0.0

Obj func = 0.5
Current x = [1]
DSP
0.5
π = 0.5
β = [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
w - 0.5 x[1] <= 0.0

Obj func = 0.5
Current x = [1]
DSP
0.5
π = 0.5
β = [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
w - 0.5 x[1] <= 0.0

Obj func = 0.5
Current x = [1]
DSP
0.5
π = 0.5
β = [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
w - 0.5 x[1] <= 0.0
x = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
y = [0.333333, 0.333333, 0.333333, 0.0, 0.333333, 0.333333, 0.333333]
